In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
idx_slice = pd.IndexSlice
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Comment out for debugging and development
warnings.simplefilter(action='ignore', category=DeprecationWarning) # Comment out for debugging and development
import plotly.graph_objects as go


from plot_helpers import (
    chdir_to_root_dir,
    read_stats_dict,
    prepare_dataframe,
    nice_title,
    load_plot_config,
    get_scen_col_function,
    save_plotly_fig)

chdir_to_root_dir()

In [ ]:
# Load configuration from YAML file
yaml_config = load_plot_config()

# Extract configuration
RUN_NAME_PREFIX = yaml_config['run']['name_prefix']
SUMMARY_VERSION = yaml_config['run']['summary_version']
SDIR = Path.cwd() / "results" / f"{RUN_NAME_PREFIX}_summary_{SUMMARY_VERSION}"
SDIR.mkdir(exist_ok=True, parents=True)

COUNTRIES = yaml_config['data']['countries']
YEARS = yaml_config['data']['years']
INDEX_LEVELS_TO_DROP = yaml_config['data']['index_levels_to_drop']

# Build index group
IDX_GROUP = idx_slice[[RUN_NAME_PREFIX], :, COUNTRIES, YEARS]
IDX_GROUP_NAME = "".join(COUNTRIES) + "_MAIN"

print(f"Index Group Name: {IDX_GROUP_NAME}")
print(f"Index Group: {IDX_GROUP}")

# Scenario filtering
SCEN_FILTER = yaml_config['data']['scen_filter']
scen_order = yaml_config['data']['scen_order']

# Get scenario column function
scen_col_func_name = yaml_config['data']['scen_col_function']
set_scen_col = get_scen_col_function(scen_col_func_name)
print(f"Using scenario column function: {scen_col_func_name}")

# Plot dimensions
width = yaml_config['plot']['width']
heights = yaml_config['plot']['heights']

# Default figure kwargs
fig_kwargs = yaml_config['plot']['default_kwargs'].copy()
fig_kwargs['width'] = width
fig_kwargs['height'] = heights['medium']

print(f"\nConfiguration loaded successfully!")
print(f"Countries: {COUNTRIES}")
print(f"Years: {YEARS}")
print(f"Scenario filter: {SCEN_FILTER if SCEN_FILTER else 'None (show all)'}")

In [ ]:
index_cols = [
    "run_name_prefix", "run_name", "country", "year", "simpl", "clusters", "ll", "opts", "sopts", "discountrate", "demand", "h2export"
 ]
mean_marginal_prices = pd.read_csv(SDIR / "load_avg_marginal_price.csv", index_col=index_cols)
mean_marginal_prices = prepare_dataframe(mean_marginal_prices, IDX_GROUP, INDEX_LEVELS_TO_DROP, set_scen_col, drop_zero=False)
mean_marginal_prices = mean_marginal_prices.query("scen in @SCEN_FILTER") if SCEN_FILTER else mean_marginal_prices

In [ ]:
mean_marginal_prices

## Marginal prices | H2 export price

In [29]:
df = mean_marginal_prices.query("variable in ['H2 export bus']").copy()


In [30]:
df["scen"] = df["scen"].str[3:]

In [ ]:
data_start="0.1MtH2export"
data_mid="0.7MtH2export"
data_end="1.0MtH2export"

In [ ]:
def marginal_price_dumbbell_fig(df, year, data_start, data_mid,data_end):

    df = df.copy()

    df = df[df.year==year]
    countries = df.sort_values(by=["value"], ascending=False)["country"]

    data = {"line_x": [], "line_y": [], 
            "data_start": [], "data_mid": [], "data_end": [], 
            "colors": [], "years": [], "countries": []}

    for country in countries:
        data["data_start"].extend([df.loc[(df.scen == data_start) & (df.country == country)]["value"].values[0]])
        data["data_mid"].extend([df.loc[(df.scen == data_mid) & (df.country == country)]["value"].values[0]])
        data["data_end"].extend([df.loc[(df.scen == data_end) & (df.country == country)]["value"].values[0]])
        data["line_x"].extend(
            [
                df.loc[(df.scen == data_start) & (df.country == country)]["value"].values[0],
                df.loc[(df.scen == data_end) & (df.country == country)]["value"].values[0],
                None,
            ]
        )
        data["line_y"].extend([country, country, None]),

    fig = go.Figure(
        data=[
            go.Scatter(
                x=data["line_x"],
                y=data["line_y"],
                mode="lines",
                showlegend=False,
                marker=dict(
                    color="grey"
                )
            ),
            go.Scatter(
                x=data["data_start"],
                y=countries,
                mode="markers+text",
                name=data_start,
                text=data["data_start"],            
                textposition="top left",
                texttemplate="%{text:.1f}",
                textfont=dict(size=11),
                marker=dict(
                    color="#A6BCC9",
                    size=13
                )

            ),
            # go.Scatter(
            #     x=data["data_mid"],
            #     y=countries,
            #     mode="markers",
            #     name=data_start,
            #     marker=dict(
            #         color="#6C9DB9",
            #         size=13
            #     )

            # ),
            go.Scatter(
                x=data["data_end"],
                y=countries,
                mode="markers+text",
                name=data_end,
                text=data["data_end"],
                textposition="top right",
                texttemplate="%{text:.1f}",
                textfont=dict(size=11),
                marker=dict(
                    color="#179c7d",
                    size=13
                )
            ),
        ]
    )

    fig.update_layout(
        title=dict(text=nice_title(f"Marginal price for H2 at export port in {year}", 
                                   "Per country and H2 export volume in €/MWh_H2_LHV")),
        width=800,
        height=500,
        legend_itemclick=False
    )
    return fig

In [ ]:
# fig = marginal_price_dumbbell_fig(df, 
#                                   year=2050, 
#                                   data_start="0.7MtH2export", 
#                                   data_mid="2.3MtH2export",
#                                   data_end="4.0MtH2export")

# fig.show()

In [ ]:
fig = marginal_price_dumbbell_fig(df, year=2035, 
                                  data_start=data_start, 
                                  data_mid=data_mid,
                                  data_end=data_end)

fig.show()